In [1]:
from vpython import *
import serial
import string
import math

from time import time

<IPython.core.display.Javascript object>

In [ ]:
# Main scene
scene=canvas(title="Arduino Nano + LSM9DS1 w/ Madgwick",
             width=500,
             height=300,
             center=vector(5,0,0),
             #range=1.2,
             forward=vector(0,-1,-0.25),
             up=vector(0,0,1),
             userpan = True)

# Second scene (Roll, Pitch, Yaw)
scene2 = canvas(title='Arduino Nano + LSM9DS1 w/ Madgwick',
                 x=0,
                 y=0,
                 width=500,
                 height=300,
                 center=vector(0,0,0),
                 background=vector(0,0,0),
                 userpan = True,
                 range=1)

# Main scene objects
# Reference axis (x,y,z)
arrow(canvas=scene,color=color.green,axis=vector(1,0,0), shaftwidth=0.05, fixedwidth=1)
arrow(canvas=scene,color=color.green,axis=vector(0,1,0), shaftwidth=0.05, fixedwidth=1)
arrow(canvas=scene,color=color.green,axis=vector(0,0,1), shaftwidth=0.05 , fixedwidth=1)
# labels
label(canvas=scene,pos=vector(0,0,0.8),text="Arduino Nano + LSM9DS1 w/ Madgwick",box=0,opacity=0)
label(canvas=scene,pos=vector(1,0,0),text="X",box=0,opacity=0)
label(canvas=scene,pos=vector(0,-1,0),text="Y",box=0,opacity=0)
label(canvas=scene,pos=vector(0,0,-1),text="Z",box=0,opacity=0)
# IMU object
platform = box(canvas=scene,up=vector(0,0,0),axis=vector(1,0,0),length=1, height=0.05, width=1, color=color.blue)
p_line = box(canvas=scene,up=vector(0,0,0),axis=vector(1,0,0),length=1,height=0.08,width=0.1,color=color.yellow)
plat_arrow = arrow(canvas=scene,up=vector(0,0,0),color=color.green,axis=vector(1,0,0), shaftwidth=0.06, fixedwidth=1)

#Second Scene
#Roll, Pitch, Yaw
cil_roll = cylinder(canvas=scene2,pos=vector(-0.4,0,0),axis=vector(0.2,0,0),radius=0.01)
cil_roll2 = cylinder(canvas=scene2,pos=vector(-0.4,0,0),axis=vector(-0.2,0,0),radius=0.01,color=color.red)
cil_pitch = cylinder(canvas=scene2,pos=vector(0.1,0,0),axis=vector(0.2,0,0),radius=0.01,color=color.green)
cil_pitch2 = cylinder(canvas=scene2,pos=vector(0.1,0,0),axis=vector(-0.2,0,0),radius=0.01,color=color.green)
#cil_course = cylinder(pos=vector(0.6,0,0),axis=(0.2,0,0),radius=0.01,color=color.blue)
#cil_course2 = cylinder(pos=vector(0.6,0,0),axis=(-0.2,0,0),radius=0.01,color=color.blue)
arrow_course = arrow(canvas=scene2,pos=vector(0.6,0,0),color=color.cyan,axis=vector(-0.2,0,0), shaftwidth=0.02, fixedwidth=1)

#Roll,Pitch,Yaw labels
label(canvas=scene2,pos=vector(-0.4,0.3,0),text="Yaw",box=0,opacity=0)
label(canvas=scene2,pos=vector(0.1,0.3,0),text="Pitch",box=0,opacity=0)
label(canvas=scene2,pos=vector(0.55,0.3,0),text="Roll",box=0,opacity=0)
label(canvas=scene2,pos=vector(0.6,0.22,0),text="N",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.6,-0.22,0),text="S",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.38,0,0),text="W",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.82,0,0),text="E",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.75,0.15,0),height=7,text="NE",box=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.45,0.15,0),height=7,text="NW",box=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.75,-0.15,0),height=7,text="SE",box=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.45,-0.15,0),height=7,text="SW",box=0,color=color.yellow)

L1 = label(canvas=scene2,pos=vector(-0.4,0.22,0),text="-",box=0,opacity=0)
L2 = label(canvas=scene2,pos=vector(0.1,0.22,0),text="-",box=0,opacity=0)
L3 = label(canvas=scene2,pos=vector(0.7,0.3,0),text="-",box=0,opacity=0)

In [ ]:
#Serial Connection for Arduino
ser = serial.Serial(port='COM3',baudrate=115200, timeout=1)

#Initialize variables
roll=0
pitch=0
yaw=0

#Degrees to radians
grad2rad = 3.141592/180.0

for i in range(0,100):
    line = ser.readline()
    #Decode bytes into string
    decoded = str(line, 'utf-8')
    #print(decoded)
    size = len(decoded)
    #slice string to remove last 2 characters from string
    mod_string = decoded[:size - 2]
    #print(mod_string)
    words = str.split(mod_string,",")    # Fields split
    print(float(words[0]),float(words[1]),float(words[2]))
    print(len(words))
    if len(words) > 2:
        #try:
        yaw = float(words[0])*grad2rad
        pitch = float(words[1])*grad2rad
        roll = float(words[2])*grad2rad
        print(yaw,pitch,roll)
        #except:
            #print("Invalid line")

        axis=vector(cos(pitch)*cos(yaw),-cos(pitch)*sin(yaw),sin(pitch)) 
        up=vector(sin(roll)*sin(yaw)+cos(roll)*sin(pitch)*cos(yaw),sin(roll)*cos(yaw)-cos(roll)*sin(pitch)*sin(yaw),-cos(roll)*cos(pitch))
        platform.axis=axis
        platform.up=up
        #platform.length=1.0
        #platform.width=0.65
        plat_arrow.axis=axis
        plat_arrow.up=up
        #plat_arrow.length=0.8
        p_line.axis=axis
        p_line.up=up
        cil_roll.axis=vector(0.2*cos(roll),0.2*sin(roll),0)
        cil_roll2.axis=vector(-0.2*cos(roll),-0.2*sin(roll),0)
        cil_pitch.axis=vector(0.2*cos(pitch),0.2*sin(pitch),0)
        cil_pitch2.axis=vector(-0.2*cos(pitch),-0.2*sin(pitch),0)
        arrow_course.axis=vector(0.2*sin(yaw),0.2*cos(yaw),0)
        L1.text = str(float(words[0]))
        L2.text = str(float(words[1]))
        L3.text = str(float(words[2]))        
ser.close